In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys
from joblib import Parallel, delayed

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

In [2]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
train_test = pd.concat([df_train, df_test], axis=0)
df_hist = utils.read_df_pkl('../input/hist_clean*')
df_new = utils.read_df_pkl('../input/new_clean*')

df_train = utils.reduce_mem_usage(df_train)
df_test  = utils.reduce_mem_usage(df_test )
df_hist  = utils.reduce_mem_usage(df_hist )
df_new   = utils.reduce_mem_usage(df_new  )

100%|██████████| 3/3 [00:01<00:00,  2.66it/s]


Mem. usage decreased to  4.04 Mb (56.2% reduction)
Mem. usage decreased to  2.24 Mb (52.5% reduction)
Mem. usage decreased to 2554.26 Mb (0.0% reduction)
Mem. usage decreased to 183.47 Mb (0.0% reduction)


In [3]:
from time_utils import date_add_days

hist_max_date = df_hist.groupby(key)['purchase_date'].max()
hist_min_date = df_hist.groupby(key)['purchase_date'].min()
hist_max_date.name = 'hist_purchase_date_max'
hist_min_date.name = 'hist_purchase_date_min'
hist_max_date = hist_max_date.to_frame()
hist_min_date = hist_min_date.to_frame()

# month_maxは翌月の数字にする
hist_max_date['hist_purchase_month_max'] = hist_max_date['hist_purchase_date_max'].map(lambda x: date_add_days(x, 32) if int(str(x)[8:10])<=15 else date_add_days(x, 20))
hist_max_date['hist_purchase_month_max'] = hist_max_date['hist_purchase_month_max'].map(lambda x: str(x)[:7])

hist_min_date['hist_purchase_month_min'] = hist_min_date['hist_purchase_date_min'].map(lambda x: str(x)[:7])


new_max_date = df_new.groupby(key)['purchase_date'].max()
new_min_date = df_new.groupby(key)['purchase_date'].min()
new_max_date.name = 'new_purchase_date_max'
new_min_date.name = 'new_purchase_date_min'
new_max_date = new_max_date.to_frame()
new_min_date = new_min_date.to_frame()

# month_maxは翌月の数字にする
new_max_date['new_purchase_month_max'] = new_max_date['new_purchase_date_max'].map(lambda x: date_add_days(x, 32) if int(str(x)[8:10])<=15 else date_add_days(x, 20))
new_max_date['new_purchase_month_max'] = new_max_date['new_purchase_month_max'].map(lambda x: str(x)[:7])
new_min_date['new_purchase_month_min'] = new_min_date['new_purchase_date_min'].map(lambda x: str(x)[:7])

df_hist.set_index(key, inplace=True)
df_new.set_index(key, inplace=True)
df_hist = df_hist.join(hist_max_date).join(hist_min_date).join(new_max_date).join(new_min_date)
df_new = df_new.join(hist_max_date).join(hist_min_date).join(new_max_date).join(new_min_date)

df_hist['hist_purchase_month_max'] = pd.to_datetime(df_hist['hist_purchase_month_max'])
df_hist['hist_purchase_month_min'] = pd.to_datetime(df_hist['hist_purchase_month_min'])
df_hist['new_purchase_month_max'] = pd.to_datetime(df_hist['new_purchase_month_max'])
df_hist['new_purchase_month_min'] = pd.to_datetime(df_hist['new_purchase_month_min'])

df_new['new_purchase_month_max'] = pd.to_datetime(df_new['new_purchase_month_max'])
df_new['new_purchase_month_min'] = pd.to_datetime(df_new['new_purchase_month_min'])
df_new['hist_purchase_month_max'] = pd.to_datetime(df_new['hist_purchase_month_max'])
df_new['hist_purchase_month_min'] = pd.to_datetime(df_new['hist_purchase_month_min'])

In [4]:
df_new_lag1 = df_new[df_new['month_lag']==1]
df_new_lag2 = df_new[df_new['month_lag']==2]

df = df_hist

auth1 = df[df.authorized_flag==1]
auth0 = df[df.authorized_flag==0]
print(auth1.shape)
print(auth0.shape)

cat1_0 = False
# cat1_0 = True
if cat1_0:
    auth1_cat1 = auth1[auth1.category_1==1]
    auth1_cat0 = auth1[auth1.category_1==0]
    auth0_cat1 = auth0[auth0.category_1==1]
    auth0_cat0 = auth0[auth0.category_1==0]
    new_cat1 = df_new[df_new.category_1==1]
    new_cat0 = df_new[df_new.category_1==0]
#     del auth1, auth0
#     gc.collect()

auth1_lag0 = auth1[auth1['month_lag']==0]
auth1_lag1 = auth1[auth1['month_lag']==-1]
auth1_lag2 = auth1[auth1['month_lag']==-2]
auth1_lag02 = auth1[auth1['month_lag']>=-2]
auth1_lag05 = auth1[auth1['month_lag']>=-5]

auth0_lag0 = auth0[auth0['month_lag']==0]
auth0_lag1 = auth0[auth0['month_lag']==-1]
auth0_lag2 = auth0[auth0['month_lag']==-2]
auth0_lag02 = auth0[auth0['month_lag']>=-2]
auth0_lag05 = auth0[auth0['month_lag']>=-5]

(26595452, 28)
(2516909, 28)


In [ ]:
prefix = '203_pst'
new_df_list = [df_new, df_new_lag1, df_new_lag2]
new_fname_list = ['new', 'new_lag1', 'new_lag2']

debug = False
if debug:
    train_test = train_test.head(10000)
        

def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]


#========================================================================
# 集計が必要なFeature
#========================================================================
for df, fname in zip(tqdm(new_df_list), new_fname_list):
    if debug:
        df = df.head(3000)
#     df.drop(['level_0', 'index'], axis=1, inplace=True)
#     sys.exit()
    
    df_train = utils.read_df_pkl('../input/train0*')
    df_test = utils.read_df_pkl('../input/test0*')
    df_train.set_index(key, inplace=True)
    df_test.set_index(key, inplace=True)
    train_test = pd.concat([df_train, df_test], axis=0)
    
    # new
    df['diff_date_from_new_min_month'] = (df['new_purchase_month_min'] - df['purchase_date']).dt.days
    df['diff_date_from_new_max_month'] = (df['new_purchase_month_max'] - df['purchase_date']).dt.days
    df['diff_date_from_hist_min_month'] = (df['hist_purchase_month_min'] - df['purchase_date']).dt.days
    df['diff_date_from_hist_max_month'] = (df['hist_purchase_month_max'] - df['purchase_date']).dt.days

    aggs = {}
#     aggs['diff_date_from_new_min_month'] = ['mean', 'max', 'min', 'std' ]
#     aggs['diff_date_from_new_max_month'] = ['mean', 'max', 'min', 'std' ]
#     aggs['diff_date_from_hist_min_month'] = ['mean', 'max', 'min', 'std']
#     aggs['diff_date_from_hist_max_month'] = ['mean', 'max', 'min', 'std']
    aggs['diff_date_from_new_min_month'] = ['skew']
    aggs['diff_date_from_new_max_month'] = ['skew']
    aggs['diff_date_from_hist_min_month'] = ['skew']
    aggs['diff_date_from_hist_max_month'] = ['skew']

#     col_unique =['subsector_id', 'merchant_id', 'merchant_category_id', 'yyyymmdd', 'yyyy_week']
#     df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
#     df['month_diff'] += df['month_lag']
    
    
#     df = pd.get_dummies(df, columns=['category_2', 'category_3'])
#     aggs = {
#     'category_1': ['sum', 'mean'],
#     'category_2_1.0': ['mean'],
#     'category_2_2.0': ['mean'],
#     'category_2_3.0': ['mean'],
#     'category_2_4.0': ['mean'],
#     'category_2_5.0': ['mean'],
#     'category_3_A': ['mean'],
#     'category_3_B': ['mean'],
#     'category_3_C': ['mean'],
#     }
    
#     for col in col_unique:
#         aggs[col] = ['nunique']

#     aggs['purchase_amount'] = ['sum','max','min','mean','var']
#     aggs['installments'] = ['sum', 'max','mean','var']
#     aggs['month_lag'] = ['max','min','mean','var']
#     aggs['month_diff'] = ['mean','var']
#     aggs['card_id'] = ['size','count']
    
    df.reset_index(inplace=True)
    
    new_columns = get_new_columns(fname, aggs)
    
    print('Aggregation Start!')
    print(df.shape)
    df_agg = df.groupby(key).agg(aggs)
    df_agg.columns = new_columns
    
    # add feature
#     df_agg[f'{fname}_amount_sum_per_installments_sum'] = df_agg[f'{fname}_purchase_amount_sum'] / (df_agg[f'{fname}_installments_sum'] + 1.0)
#     df_agg[f'{fname}_amount_mean_per_installments_mean'] = df_agg[f'{fname}_purchase_amount_mean'] / (df_agg[f'{fname}_installments_mean'] + 1.0)
    
    
    #========================================================================
    # monthly agg
    #========================================================================
    new_columns = get_new_columns(fname + '_monthly_avg', aggs)
    month_agg = df.groupby([key, 'yyyymm']).agg(aggs)
        
    month_agg.columns = new_columns
    month_agg = month_agg.reset_index().drop('yyyymm', axis=1).groupby([key]).mean()
    
#     train_test = train_test.join(df_agg)
    train_test = train_test.join(df_agg).join(month_agg)
    print(train_test.shape)
    
    #========================================================================
    # Save Feature
    ignore_features = ['first_active_month', 'card_id', target]
    for col in train_test.columns:
        if col in ignore_features: continue
        if col.count('feature_'):continue
        feature = train_test[col].astype('float32').values
        col = col.replace('.', '_')
    #     utils.to_pkl_gzip(path = f'../features/1_first_valid/{prefix}_{col}@', obj=feature)
        utils.to_pkl_gzip(path = f'../features/3_third_valid/{prefix}_{col}@', obj=feature)
    #========================================================================


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 92.12it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 160.99it/s]

Aggregation Start!
(1963031, 35)
(325540, 13)



 33%|███▎      | 1/3 [07:09<14:18, 429.50s/it]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 60.69it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 109.49it/s]

Aggregation Start!
(1027617, 34)
(325540, 13)



 67%|██████▋   | 2/3 [12:09<06:30, 390.51s/it]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 106.57it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 159.15it/s]

Aggregation Start!
(935414, 34)
(325540, 13)



100%|██████████| 3/3 [16:35<00:00, 353.27s/it]


In [ ]:
hist_df_list = [auth1, auth1_lag0, auth1_lag1, auth1_lag2, auth1_lag02, auth1_lag05, auth0, auth0_lag0, auth0_lag1, auth0_lag2, auth0_lag02, auth0_lag05] 
hist_fname_list = ['auth1', 'auth1_lag0', 'auth1_lag1', 'auth1_lag2', 'auth1_lag02', 'auth1_lag05', 'auth0', 'auth0_lag0', 'auth0_lag1', 'auth0_lag2', 'auth0_lag02', 'auth0_lag05']

for df, fname in zip(tqdm(hist_df_list), hist_fname_list):
    if debug:
        df = df.head(3000)
    
    df_train = utils.read_df_pkl('../input/train0*')
    df_test = utils.read_df_pkl('../input/test0*')
    df_train.set_index(key, inplace=True)
    df_test.set_index(key, inplace=True)
    train_test = pd.concat([df_train, df_test], axis=0)
    
#     hist
    print('Personal Time Feature.')
    df['diff_date_from_new_min_month'] = (df['new_purchase_month_min'] - df['purchase_date']).dt.days
    df['diff_date_from_new_max_month'] = (df['new_purchase_month_max'] - df['purchase_date']).dt.days
    df['diff_date_from_hist_min_month'] = (df['hist_purchase_month_min'] - df['purchase_date']).dt.days
    df['diff_date_from_hist_max_month'] = (df['hist_purchase_month_max'] - df['purchase_date']).dt.days
    
    aggs = {}
    
#     df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
#     df['month_diff'] += df['month_lag']

#     col_unique =['subsector_id', 'merchant_id', 'merchant_category_id', 'yyyymmdd', 'yyyy_week']

#     df.reset_index(inplace=True)
#     df = pd.get_dummies(df, columns=['category_2', 'category_3'])
#     aggs = {
#     'category_2_1.0': ['mean'],
#     'category_2_2.0': ['mean'],
#     'category_2_3.0': ['mean'],
#     'category_2_4.0': ['mean'],
#     'category_2_5.0': ['mean'],
#     'category_3_A': ['mean'],
#     'category_3_B': ['mean'],
#     'category_3_C': ['mean'],
#     }
    
#     for col in col_unique:
#         aggs[col] = ['nunique']

#     aggs['diff_date_from_new_min_month'] = ['mean', 'max', 'min', 'std']
#     aggs['diff_date_from_new_max_month'] = ['mean', 'max', 'min', 'std']
#     aggs['diff_date_from_hist_min_month'] = ['mean', 'max', 'min', 'std']
#     aggs['diff_date_from_hist_max_month'] = ['mean', 'max', 'min', 'std']
    aggs['diff_date_from_new_min_month'] = ['skew']
    aggs['diff_date_from_new_max_month'] = ['skew']
    aggs['diff_date_from_hist_min_month'] = ['skew']
    aggs['diff_date_from_hist_max_month'] = ['skew']
    
#     aggs['category_1'] = ['sum', 'mean']
#     aggs['purchase_amount'] = ['sum','max','min','mean','var']
#     aggs['installments'] = ['sum','max','mean','var']
#     aggs['month_lag'] = ['max','min','mean','var']
#     aggs['month_diff'] = ['mean','var']
#     aggs['card_id'] = ['size','count']
    
    new_columns = get_new_columns(fname, aggs)
    df_agg = df.groupby(key).agg(aggs)
    df_agg.columns = new_columns
    
    # add feature
#     df_agg[f'{fname}_amount_sum_per_installments_sum'] = df_agg[f'{fname}_purchase_amount_sum'] / (df_agg[f'{fname}_installments_sum'] + 1.0)
#     df_agg[f'{fname}_amount_mean_per_installments_mean'] = df_agg[f'{fname}_purchase_amount_mean'] / (df_agg[f'{fname}_installments_mean'] + 1.0)
    
    #========================================================================
    # monthly agg
    #========================================================================
    new_columns = get_new_columns(fname + '_monthly_avg', aggs)
    month_agg = df.groupby([key, 'yyyymm']).agg(aggs)
        
    month_agg.columns = new_columns
    month_agg = month_agg.reset_index().drop('yyyymm', axis=1).groupby([key]).mean()
    
    train_test = train_test.join(df_agg).join(month_agg)
    print(train_test.shape)
    
    #========================================================================
    # Save Feature
    ignore_features = ['first_active_month', 'card_id', target]
    for col in train_test.columns:
        if col in ignore_features: continue
        if col.count('feature_'):continue
        feature = train_test[col].astype('float32').values
        col = col.replace('.', '_')
#         utils.to_pkl_gzip(path = f'../features/1_first_valid/{prefix}_{col}@', obj=feature)
        utils.to_pkl_gzip(path = f'../features/3_third_valid/{prefix}_{col}@', obj=feature)
    #========================================================================    


  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 42.53it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 58.00it/s]

Personal Time Feature.
(325540, 13)



  8%|▊         | 1/12 [24:29<4:29:19, 1469.01s/it]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 110.85it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 174.40it/s]

Personal Time Feature.


In [9]:
import glob
# #========================================================================
# # これは作成済のFeatureを読み込んで計算する
# #========================================================================
combi_list = [ ['new', 'new_lag1'] ,['new', 'auth1'] ,['new', 'auth1_lag0'] ,['new', 'auth1_lag02'] ,['new', 'auth1_lag05'] ,['new', 'auth0_lag0'] ,
['new', 'auth0_lag02'] ,['new', 'auth0_lag05'] ,['auth1_lag0', 'auth1_lag02'] ,['auth1_lag0', 'auth1_lag05'] ,['auth1_lag02', 'auth1_lag05'] ,['auth0_lag02', 'auth0_lag05']
]

comp_cols = [
#     'amount_sum_per_installments_sum'
#     ,'amount_mean_per_installments_mean'
#     ,'purchase_amount_sum' 
#     ,'purchase_amount_mean' 
#     ,'purchase_amount_max' 
#     ,'purchase_amount_min' 
#     ,'installments_max' 
#     ,'installments_var' 
#     ,'category_1_sum' 
#     ,'category_1_mean' 
#     ,'month_lag_mean' 
#     ,'month_lag_var' 
#     ,'month_lag_skew' 
#     ,'month_diff_mean' 
#     ,'month_diff_var' 
#     ,'month_diff_skew' 
#     ,'card_id_size' 
#     ,'yyyymmdd_nunique'
#     ,'subsector_id_nunique'
#     ,'merchant_id_nunique'
#     ,'merchant_category_id_nunique'
     'diff_date_from_new_min_month_mean'
     ,'diff_date_from_hist_max_month_mean'
     ,'diff_date_from_new_min_month_max'
     ,'diff_date_from_hist_max_month_max'
     ,'diff_date_from_new_min_month_min'
     ,'diff_date_from_hist_min_month_min'
]

tmp_feature_list = glob.glob('../features/3_third_valid/203_*.gz')
feature_list = []
for f in tmp_feature_list:
    if f.count('pst_ratio_') or f.count('pst_diff_'):continue
    for col in comp_cols:
        if f.count(col):
            feature_list.append(f)
            
base = utils.read_df_pkl('../input/base_first*0*')
p_list = utils.parallel_load_data(path_list=feature_list)
df_feat = pd.concat(p_list, axis=1)
train_test = pd.concat([base[key], df_feat], axis=1)

for (fm1, fm2) in tqdm(combi_list):
    for col in comp_cols:
#         203_pst_auth0_lag02_monthly_avg_purchase_amount_min
        try:
            train_test[f"ratio_{fm1}_{fm2}_{col}"] = train_test[f"{prefix}_{fm1}_{col}@"] / train_test[f"{prefix}_{fm2}_{col}@"]
            train_test[f"diff_{fm1}_{fm2}_{col}"] = train_test[f"{prefix}_{fm1}_{col}@"] - train_test[f"{prefix}_{fm2}_{col}@"]
        except KeyError:
            print(fm1, fm2, col)
            continue
print(train_test.shape)
train_test.head()


100%|██████████| 3/3 [00:00<00:00, 26.74it/s]

100%|██████████| 12/12 [00:00<00:00, 21.35it/s]

(325540, 325)


,card_id,203_pst_new_diff_date_from_hist_min_month_min@,203_pst_auth1_lag1_diff_date_from_new_min_month_max@,203_pst_new_lag1_monthly_avg_diff_date_from_new_min_month_min@,203_pst_auth1_lag2_diff_date_from_hist_min_month_min@,203_pst_auth1_lag1_monthly_avg_diff_date_from_hist_max_month_mean@,203_pst_auth1_lag1_monthly_avg_diff_date_from_new_min_month_max@,203_pst_auth0_lag0_diff_date_from_hist_max_month_max@,203_pst_auth0_lag1_monthly_avg_diff_date_from_hist_min_month_min@,203_pst_auth0_lag0_monthly_avg_diff_date_from_hist_min_month_min@,203_pst_auth0_lag05_monthly_avg_diff_date_from_hist_max_month_max@,203_pst_auth1_lag0_monthly_avg_diff_date_from_hist_max_month_mean@,203_pst_auth0_lag02_monthly_avg_diff_date_from_hist_max_month_mean@,203_pst_auth0_lag02_diff_date_from_new_min_month_mean@,203_pst_auth1_lag05_diff_date_from_new_min_month_min@,203_pst_auth1_lag05_monthly_avg_diff_date_from_hist_max_month_max@,203_pst_auth1_lag2_monthly_avg_diff_date_from_new_min_month_mean@,203_pst_auth0_monthly_avg_diff_date_from_hist_min_month_min@,203_pst_auth1_lag02_diff_date_from_new_min_month_max@,203_pst_auth1_lag0_monthly_avg_diff_date_from_new_min_month_max@,203_pst_auth0_monthly_avg_diff_date_from_hist_max_month_max@,203_pst_auth1_lag1_diff_date_from_new_min_month_min@,203_pst_auth1_lag0_diff_date_from_new_min_month_mean@,203_pst_auth0_lag1_monthly_avg_diff_date_from_new_min_month_max@,203_pst_auth0_monthly_avg_diff_date_from_hist_max_month_mean@,203_pst_auth0_lag2_monthly_avg_diff_date_from_hist_max_month_max@,203_pst_auth0_lag1_diff_date_from_hist_max_month_max@,203_pst_auth1_diff_date_from_new_min_month_min@,203_pst_auth0_diff_date_from_hist_max_month_mean@,203_pst_auth1_lag2_monthly_avg_diff_date_from_hist_min_month_min@,203_pst_auth1_diff_date_from_hist_min_month_min@,203_pst_auth1_diff_date_from_new_min_month_mean@,203_pst_auth1_lag02_diff_date_from_hist_max_month_max@,203_pst_auth1_lag0_monthly_avg_diff_date_from_new_min_month_mean@,203_pst_auth1_lag02_monthly_avg_diff_date_from_new_min_month_max@,203_pst_auth1_lag05_diff_date_from_hist_min_month_min@,203_pst_auth0_lag0_diff_date_from_new_min_month_max@,203_pst_auth1_lag2_monthly_avg_diff_date_from_hist_max_month_max@,203_pst_auth0_lag2_monthly_avg_diff_date_from_hist_min_month_min@,203_pst_auth1_lag0_diff_date_from_hist_min_month_min@,203_pst_auth1_lag02_diff_date_from_hist_min_month_min@,203_pst_auth0_lag02_diff_date_from_new_min_month_min@,203_pst_auth1_diff_date_from_hist_max_month_max@,203_pst_new_lag1_diff_date_from_new_min_month_min@,203_pst_auth0_lag02_diff_date_from_hist_max_month_mean@,203_pst_new_lag2_monthly_avg_diff_date_from_hist_max_month_max@,203_pst_auth1_lag05_monthly_avg_diff_date_from_new_min_month_mean@,203_pst_new_lag1_diff_date_from_new_min_month_mean@,203_pst_auth1_lag02_monthly_avg_diff_date_from_new_min_month_min@,203_pst_auth1_monthly_avg_diff_date_from_hist_max_month_max@,203_pst_auth0_lag1_diff_date_from_new_min_month_max@,203_pst_auth0_lag1_diff_date_from_new_min_month_min@,203_pst_auth0_lag1_monthly_avg_diff_date_from_new_min_month_min@,203_pst_auth1_lag02_monthly_avg_diff_date_from_hist_max_month_mean@,203_pst_new_lag2_diff_date_from_new_min_month_min@,203_pst_new_lag2_diff_date_from_hist_max_month_max@,203_pst_new_monthly_avg_diff_date_from_hist_min_month_min@,203_pst_new_lag2_monthly_avg_diff_date_from_new_min_month_mean@,203_pst_new_lag2_monthly_avg_diff_date_from_new_min_month_min@,203_pst_auth1_lag05_diff_date_from_hist_max_month_max@,203_pst_auth0_lag05_diff_date_from_hist_max_month_mean@,203_pst_auth0_lag1_diff_date_from_new_min_month_mean@,203_pst_auth1_lag02_diff_date_from_new_min_month_min@,203_pst_auth0_lag02_diff_date_from_hist_min_month_min@,203_pst_auth1_lag05_monthly_avg_diff_date_from_new_min_month_max@,203_pst_auth0_lag0_diff_date_from_hist_min_month_min@,203_pst_auth0_diff_date_from_hist_max_month_max@,203_pst_auth1_lag0_monthly_avg_diff_date_from_new_min_month_min@,203_pst_auth0_lag1_diff_date_from_hist_max_month_mean@,203_pst_

In [10]:
### Save Feature
# train_test = train_test.join(hist_max_date).join(hist_min_date).join(new_max_date).join(new_min_date)
# train_test['hist_purchase_month_max'] = pd.to_datetime(train_test['hist_purchase_month_max'])
# train_test['hist_purchase_month_min'] = pd.to_datetime(train_test['hist_purchase_month_min'])
# train_test['new_purchase_month_max'] = pd.to_datetime(train_test['new_purchase_month_max'])
# train_test['new_purchase_month_min'] = pd.to_datetime(train_test['new_purchase_month_min'])
# train_test.head()

# # 静的なFeature
# train_test['personal_term_new_min_hist_max'] = (train_test['new_purchase_month_min'] - train_test['hist_purchase_month_max']).dt.days
# train_test['personal_term_new_min_hist_min'] = (train_test['new_purchase_month_min'] - train_test['hist_purchase_month_min']).dt.days
# train_test['personal_days_new_max_date_from_hist_max_month'] = (train_test['new_purchase_date_max'] - train_test['hist_purchase_month_max']).dt.days
# train_test['personal_days_new_min_date_from_hist_max_month'] = (train_test['new_purchase_date_min'] - train_test['hist_purchase_month_max']).dt.days
        
# prefix = '203_pst'
# ignore_features = ['first_active_month', 'card_id', target]

for col in train_test.columns:
    if col in ignore_features: continue
    if col.count('203_'):continue
    if not(col.count('ratio')) and not(col.count('diff_')):continue
    feature = train_test[col].astype('float32').values
    if np.std(feature)==0:
        continue
    
#     utils.to_pkl_gzip(path = f'../features/1_first_valid/{prefix}_{col}@', obj=feature)
    utils.to_pkl_gzip(path = f'../features/3_third_valid/{prefix}_{col}@', obj=feature)